In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"

import pathlib
import glob
from copy import deepcopy

import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
os.environ["JAX_PLATFORM_NAME"] = "cpu"

import jax
import jax.numpy as jnp
import equinox as eqx

# gpus = jax.devices()
# jax.config.update("jax_enable_x64", False)
#jax.config.update("jax_default_device", gpus[1])
jax.config.update("jax_platform_name", "cpu")

In [2]:
from mc2.utils.data_inspection import (
    get_available_material_names,
    get_file_overview,
    load_and_process_single_from_full_file_overview
)

from mc2.data_management import FrequencySet, MaterialSet, DataSet, NormalizedFrequencySet,load_data_into_pandas_df

In [3]:
data_dict = load_data_into_pandas_df(material="3C90")
mat_set = MaterialSet.from_pandas_dict(data_dict)
train_set, val_set, test_set = mat_set.split_into_train_val_test(
    train_frac=0.7, val_frac=0.15, test_frac=0.15, seed=12
)

Loading data for 3C90: 100%|██████████| 21/21 [00:12<00:00,  1.67it/s]
INFO:2025-08-06 14:26:01,423:jax._src.xla_bridge:925: Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'


2025-08-06 14:26:01 | INFO : Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'


INFO:2025-08-06 14:26:01,427:jax._src.xla_bridge:925: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


2025-08-06 14:26:01 | INFO : Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


In [18]:
from mc2.features.features_jax import compute_fe_single
def featurize(norm_B_past, norm_H_past, norm_B_future, temperature, frequency):
    past_length = norm_B_past.shape[0]
    future_length = norm_B_future.shape[0]
    
    featurized_B = compute_fe_single(jnp.hstack([norm_B_past, norm_B_future]), n_s=10)

    return featurized_B[past_length:]

feature_names=["original_b","db_dt","d2b_dt2","dyn_avg","pwm_of_b"]

In [ ]:
train_set_norm = train_set.normalize(transform_H=True, featurize=featurize) #  ,feature_names=feature_names

In [ ]:
norm_dict={}
for material_name in ["3C90","3C94","3E6","3F4","77","78","N27","N30","N87"]:
    data_dict = load_data_into_pandas_df(material=material_name)
    mat_set = MaterialSet.from_pandas_dict(data_dict)
    train_set, val_set, test_set = mat_set.split_into_train_val_test(
        train_frac=0.7, val_frac=0.15, test_frac=0.15, seed=12
    )
    train_set_norm = train_set.normalize(transform_H=True, featurize=featurize)
    norm_dict_basic= {  "B_max": train_set_norm.normalizer.B_max,
                        "H_max": train_set_norm.normalizer.H_max,
                        "T_max": train_set_norm.normalizer.T_max,
                        "f_max": train_set_norm.normalizer.f_max}
    features_dict= {}
    for feature_name, feature_value in zip(feature_names, train_set_norm.normalizer.norm_fe_max):
        features_dict[feature_name] = float(feature_value)
    norm_dict_basic["features"] = features_dict
    norm_dict_train = deepcopy(norm_dict_basic)

    mat_set_norm = mat_set.normalize(transform_H=True, featurize=featurize)
    norm_dict_basic= {  "B_max": mat_set_norm.normalizer.B_max,
                        "H_max": mat_set_norm.normalizer.H_max,
                        "T_max": mat_set_norm.normalizer.T_max,
                        "f_max": mat_set_norm.normalizer.f_max}
    features_dict= {}
    for feature_name, feature_value in zip(feature_names, mat_set_norm.normalizer.norm_fe_max):
        features_dict[feature_name] = float(feature_value)
    norm_dict_basic["features"] = features_dict
    norm_dict_complete= deepcopy(norm_dict_basic)

    norm_dict[material_name] = {"complete_set":norm_dict_complete,"train_set":norm_dict_train}

    del mat_set, train_set, val_set, test_set, train_set_norm, mat_set_norm

Loading data for N87: 100%|██████████| 21/21 [00:11<00:00,  1.78it/s]


In [56]:
from pathlib import Path
import os
import json

file_path = os.path.join(os.getcwd())
path = Path(file_path).parent.parent / "data" / "normalization_values"

file_path_json = path / "norm_values.json"

with open(file_path_json, "w") as f:
    json.dump(norm_dict, f, indent=2)